# Cramer GAN implementation by tensorflow

In [ ]:
import tensorflow as tf
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# utility func

In [ ]:
# save metrics
def save_metrics(metrics, epoch=None):
    # make directory if there is not
    path = "metrics"
    if not os.path.isdir(path):
        os.makedirs(path)

    # save metrics
    plt.figure(figsize=(10,8))
    plt.plot(metrics["d_loss"], label="discriminative loss", color="b")
    plt.legend()
    plt.savefig(os.path.join(path, "dloss" + str(epoch) + ".png"))
    plt.close()

    plt.figure(figsize=(10,8))
    plt.plot(metrics["g_loss"], label="generative loss", color="r")
    plt.legend()
    plt.savefig(os.path.join(path, "g_loss" + str(epoch) + ".png"))
    plt.close()

    plt.figure(figsize=(10,8))
    plt.plot(metrics["g_loss"], label="generative loss", color="r")
    plt.plot(metrics["d_loss"], label="discriminative loss", color="b")
    plt.legend()
    plt.savefig(os.path.join(path, "both_loss" + str(epoch) + ".png"))
    plt.close()

In [ ]:
# plot images
def save_imgs(images, plot_dim=(10,6), size=(6,10), epoch=None):
    # make directory if there is not
    path = "generated_figures"
    if not os.path.isdir(path):
        os.makedirs(path)

    num_examples = plot_dim[0]*plot_dim[1]
    num_examples = 60
    fig = plt.figure(figsize=size)

    for i in range(num_examples):
        plt.subplot(plot_dim[0], plot_dim[1], i+1)
        img = images[i, :]
        img = img.reshape((32, 32, 3))
        plt.tight_layout()
        plt.imshow(img)
        plt.axis("off")
    plt.subplots_adjust(wspace=0.1, hspace=0.1)
    plt.savefig(os.path.join(path, str(epoch) + ".png"))
    plt.close()

In [ ]:
# training
import pickle
import numpy as np
import os

def unpickle(file):
    fo = open(file, 'rb')
    #print(file)
    dict = pickle.load(fo, encoding='latin1')
    fo.close()
    return dict

def one_hot_vec(label):
    vec = np.zeros(10)
    vec[label] = 1
    return vec

def load_data():
    x_all = []
    y_all = []
    for i in range (5):
        d = unpickle("cifar-10-batches-py/data_batch_" + str(i+1))
        x_ = d['data']
        y_ = d['labels']
        x_all.append(x_)
        y_all.append(y_)

    d = unpickle('cifar-10-batches-py/test_batch')
    x_all.append(d['data'])
    y_all.append(d['labels'])

    x = np.concatenate(x_all) / np.float32(255)
    y = np.concatenate(y_all)
    x = np.dstack((x[:, :1024], x[:, 1024:2048], x[:, 2048:]))
    x = x.reshape((x.shape[0], 32, 32, 3))

    #pixel_mean = np.mean(x[0:50000],axis=0)
    #x -= pixel_mean
    y = np.array(list(map(one_hot_vec, y)))
    #X_train = x[0:50000,:,:,:]
    #Y_train = y[0:50000]
    #X_test = x[50000:,:,:,:]
    #Y_test = y[50000:]

    #return (X_train, Y_train, X_test, Y_test)
    return x, y

# model

In [ ]:
class Generator:
    def __init__(self):
        self.reuse = False
        self.initializer = tf.contrib.layers.xavier_initializer()
        self.X_dim = 32*32*3
        self.z_dim = 100
        self.y_dim = 10

        self.depths = [1024, 512, 256, 128, 3]
        self.s_size = 6

    def __call__(self, z, y, training=False):
        with tf.variable_scope('g', reuse=self.reuse):
            #print(z.get_shape())
            #print(y.get_shape())
            inputs = tf.concat([z, y], 1)
            inputs = tf.reshape(inputs, [-1, self.z_dim + self.y_dim])
            fc1 = inputs
#            fc1 = tf.layers.dense(inputs, 1024)
#            fc1 = tf.nn.relu(fc1)
            fc2 = tf.layers.dense(fc1, 2*2*self.depths[0])
            fc2 = tf.reshape(fc2, [-1, 2,2,self.depths[0]])
            fc2 = tf.nn.relu(fc2)
            conv1 = tf.contrib.layers.conv2d_transpose(fc2, self.depths[1], [4,4],[2,2]) # 4
            conv1 = tf.nn.relu(conv1)
            conv2 = tf.contrib.layers.conv2d_transpose(conv1, self.depths[2], [4,4],[2,2]) # 8
            conv2 = tf.nn.relu(conv2)
            conv3 = tf.contrib.layers.conv2d_transpose(conv2, self.depths[3], [4,4],[2,2]) # 16
            conv3 = tf.nn.relu(conv3)
            conv4 = tf.contrib.layers.conv2d_transpose(conv3, self.depths[4], [4,4],[2,2], activation_fn=tf.sigmoid)
            conv4 = tf.reshape(conv4, [-1, 32*32*3])
            outputs = conv4

        self.reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='g')
        return outputs

In [ ]:
class Discriminator:
    def __init__(self):
        self.reuse = False
        self.X_dim = 32*32*3
        self.initializer = tf.contrib.layers.xavier_initializer()

        self.depths = [3, 64, 128, 256, 512]
        self.s_size = 6

    def __call__(self, x, y, training=False, name=''):
        def leaky_relu(x, leak=0.2, name='outputs'):
            return tf.maximum(x, x * leak, name=name)

        with tf.name_scope('d' + name), tf.variable_scope('d', reuse=self.reuse):
            x = tf.reshape(x, [-1, 32*32*3])
            y = tf.reshape(y, [-1, 10])
            inputs = tf.concat([x, y], 1)
            inputs = tf.layers.dense(inputs, 32*32*3)
            x = tf.reshape(inputs, [-1, 32, 32, 3])
            conv1 = tf.layers.conv2d(x, self.depths[1], [4,4], [2,2]) # 16
            conv1 = leaky_relu(conv1)
            conv2 = tf.layers.conv2d(conv1, self.depths[2], [4,4], [2,2]) # 8
            conv2 = leaky_relu(conv2)
            conv3 = tf.layers.conv2d(conv2, self.depths[3], [4,4], [2,2]) # 4
            conv3 = leaky_relu(conv3)
            #conv4 = tf.layers.conv2d(conv3, self.depths[4], [4,4], [2,2]) # 2
            #conv4 = leaky_relu(conv4)
            conv4 = tf.contrib.layers.flatten(conv3) 
            fc1 = tf.layers.dense(conv4, 512)
            fc1 = leaky_relu(fc1)
            fc2 = tf.layers.dense(fc1, 1)
            """
            # convolution x 4
            outputs = tf.reshape(inputs, [-1, 96, 96, 3])

            with tf.variable_scope('conv1'):
                outputs = tf.layers.conv2d(outputs, self.depths[1], [5, 5], strides=(2, 2), padding='SAME')
                outputs = leaky_relu(outputs, name='outputs')
            with tf.variable_scope('conv2'):
                outputs = tf.layers.conv2d(outputs, self.depths[2], [5, 5], strides=(2, 2), padding='SAME')
                outputs = leaky_relu(outputs, name='outputs')
            with tf.variable_scope('conv3'):
                outputs = tf.layers.conv2d(outputs, self.depths[3], [5, 5], strides=(2, 2), padding='SAME')
                outputs = leaky_relu(outputs, name='outputs')
            with tf.variable_scope('conv4'):
                outputs = tf.layers.conv2d(outputs, self.depths[4], [5, 5], strides=(2, 2), padding='SAME')
                outputs = leaky_relu(outputs, name='outputs')
            with tf.variable_scope('classify'):
                outputs = tf.layers.dense(outputs, 256, name='outputs')
            """
        outputs = fc2
        self.reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='d')
        return outputs

In [ ]:
# f(x) = ||h(x) - h(x_)|| - ||h(x)||
class Critic(object):
    def __init__(self, h):
        self.h = h
    def __call__(self, x, x_):
        return tf.norm(self.h(x) - self.h(x_), axis=1) - tf.norm(self.h(x), axis=1)

# f(x) = ||h(x) - h(x_)||
class calc_Norm(object):
    def __init__(self, h):
        self.h = h
    def __call__(self, x, x_):
        return tf.norm(self.h(x) -self.h(x_))

In [ ]:
def conv_concat(x, y):
    batch_size = tf.shape(x)[0]
    y = tf.reshape(y, [batch_size, 1, 1, 10])
    return tf.concat([x, y], 1) # bzx32x32x3 + bzx1x1x10

class GAN:
    def __init__(self):
        self.batch_size = 256
        img_size = 32
        num_class = 10
        self.z_dim = 100

        self.epochs = 1000000
        self.epoch_saveMetrics = 500
        self.epoch_saveSampleImg = 500
        self.epoch_saveParamter = 10000
        self.losses = {"d_loss":[], "g_loss":[]}

        self.x, self.y = load_data()

        self.X_tr = tf.placeholder(tf.float32, shape=[None, img_size, img_size, 3])
        self.Y_tr = tf.placeholder(tf.float32, shape=[None, num_class])
        self.Z1 = tf.placeholder(tf.float32, [None, self.z_dim])
        
        self.g = Generator()
        self.d = Discriminator()
        self.Xg = self.g(self.Z1, self.Y_tr)

    def loss(self, Xr, Yr, z1):
        Xg = self.g(z1, Yr)
        Xg = tf.reshape(Xg, [self.batch_size, -1]) 
        Xr = tf.reshape(Xr, [self.batch_size, -1])
        epsilon = tf.random_uniform([], 0.0, 1.0) # 怪しい
        Xhat = epsilon * Xr + (1-epsilon) * Xg   
        dhat = self.d(Xhat, Yr)
        ddx = tf.gradients(dhat, Xhat)[0]
        #print((ddx.get_shape().as_list()))
        ddx = tf.norm(ddx, axis=1)
        ddx = tf.reduce_mean(tf.square(ddx - 1.0) * 10)      
  
        L_d = self.d(Xg, Yr) - self.d(Xr, Yr) + ddx
        L_g = -self.d(Xg, Yr)
        return L_g, L_d

    def train(self):
        # Optimizer
        learning_rate = 1e-4
        beta1 = 0.5
        beta2 = 0.9

        self.L_g, self.L_d = self.loss(Xr=self.X_tr, Yr=self.Y_tr, z1=self.Z1)

        d_opt = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=beta1, beta2=beta2)
        d_train_op = d_opt.minimize(self.L_d, var_list=self.d.variables)
        g_opt = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=beta1, beta2=beta2)
        g_train_op = g_opt.minimize(self.L_g, var_list=self.g.variables)

        saver = tf.train.Saver()
        #%debug
        
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            #rom tensorflow.python import debug as tf_debug
            #ess = tf_debug.LocalCLIDebugWrapperSession(sess)

            z_test = np.random.uniform(-1, 1, size=[60, self.z_dim])
            y_tmp = np.repeat(np.arange(10), 6)
            y_test = np.array(list(map(one_hot_vec, y_tmp)))
            print(y_test.shape)
            print(y_test)

            for epoch in range(self.epochs):

                # X_mb, Y_mbを収集
                def extractXimg(path, batch_size):
                    imgs = os.listdir(path)
                    rand_id = np.random.randint(0, len(imgs), size=batch_size)
                    X_mb = np.zeros((1, 32, 32, 3))

                    for i in range(batch_size):
                        img = Image.open(path+"/"+imgs[rand_id[i]])
                        img_np = np.asarray(img)
                        X_mb = np.vstack((X_mb, img_np[np.newaxis, :]))
                        img.close()
                    X_mb = X_mb[1:,:]
                    return X_mb
                
                for _ in range(5):
                    # 訓練データを抜粋
                    z1 = np.random.uniform(-1, 1, size=[self.batch_size, self.z_dim])

                    rand_id = np.random.randint(0, self.x.shape[0], size=self.batch_size)
                    X_mb = self.x[rand_id]
                    Y_mb = self.y[rand_id]

                    # X_mb = X_mb/255
                    X_mb = X_mb.astype(np.float32)

                    # Y_mb = Y_mb/255
                    Y_mb = Y_mb.astype(np.float32)
                    
                    _, d_loss_value = sess.run([d_train_op, self.L_d], feed_dict={self.X_tr: X_mb, self.Y_tr: Y_mb, self.Z1:z1})

                # train G
                _, g_loss_value = sess.run([g_train_op, self.L_g], feed_dict={self.X_tr: X_mb, self.Y_tr: Y_mb, self.Z1: z1})

                # generate Sample Imgs
                #sampleImgsOfX2Y, sampleImgsOfY2X = sess.run([self.X2Y, self.Y2X], feed_dict={self.X_tr: X_mb, self.Y_tr: Y_mb})

                # 結果をappend
                self.losses["d_loss"].append(np.sum(d_loss_value))
                self.losses["g_loss"].append(np.sum(g_loss_value))
                
                if epoch % 100 == 0:
                    print("epoch:" + str(epoch))

                # lossの可視化
                if epoch % self.epoch_saveMetrics == 1:
                    save_metrics(self.losses, epoch)

                # 画像の変換テスト
                if epoch % self.epoch_saveSampleImg == 0:
                    img = sess.run(self.Xg, feed_dict={self.Z1: z_test, self.Y_tr: y_test})
                    save_imgs(img, epoch=str(epoch))
                    # imgs_Y = np.vstack((sampleImgsOfX2Y, Y_mb))
                    # imgs_X = np.vstack((sampleImgsOfY2X, X_mb))
                    
                    # save_imgs(imgs_Y, epoch=str(epoch)+"X2Y")
                    #save_imgs(imgs_X, epoch=str(epoch)+"Y2X")
                    
                    #save_imgs(sampleImgsOfX2Y, epoch=str(epoch)+"X2Y")
                    #save_imgs(sampleImgsOfY2X, epoch=str(epoch)+"Y2X")

                # parameterのsave
                if epoch % self.epoch_saveParamter == 1:
                    path = "model"
                    if not os.path.isdir(path):
                        os.makedirs(path)

                    saver.save(sess, "./model/dcgan_model" + str(epoch) + ".ckpt")
       

    def sample_images(self, row=5, col=12, inputs=None, epoch=None):
        images = self.g(inputs, training=True)
        return images

In [ ]:
gan = GAN()
gan.train()

In [ ]:
import numpy as np

X_train = np.load("irasutoya_face_1813x96x96x3_jpg.npy")
X_train = X_train/255
X_dim = 96*96*3
z_dim = 100
batch_size = 32
epochs = 500000
display_epoch = 100
param_save_epoch = 10000
loss = {"dis_loss":[], "gen_loss":[]}

# main func

In [ ]:
p_noise = tf.placeholder(tf.float32, [None, z_dim])
noise_check = np.random.uniform(-1, 1, size=[60, z_dim]).astype(np.float32)

Xr = tf.placeholder(tf.float32, [None, X_dim])
Z1 = tf.placeholder(tf.float32, [None, z_dim])
#Z2 = tf.placeholder(tf.float32, [None, z_dim])

gan = GAN()

losses = gan.loss(Xr=Xr, z1=Z1)
d_train_op = gan.d_train(losses)
g_train_op = gan.g_train(losses)

saver = tf.train.Saver()
#%debug
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(epochs):
        if epoch % display_epoch == 0:
            imgs_gen = gan.sample_images(inputs=noise_check).eval()
            print("saving images")
            save_imgs(imgs_gen, epoch=epoch)

        for _ in range(5):
            # 訓練データを抜粋
            rand_index = np.random.randint(0, X_train.shape[0], size=batch_size)
            X_mb = X_train[rand_index, :].astype(np.float32)
            X_mb = X_mb.reshape([-1, X_dim])

            z1 = np.random.uniform(-1, 1, size=[batch_size, z_dim])
            _, d_loss_value = sess.run([d_train_op, losses[gan.d]], feed_dict={Xr: X_mb, Z1:z1})


        # 訓練データを抜粋
        rand_index = np.random.randint(0, X_train.shape[0], size=batch_size)
        X_mb = X_train[rand_index, :].astype(np.float32)
        X_mb = X_mb.reshape([-1, X_dim])
        z1 = np.random.uniform(-1, 1, size=[batch_size, z_dim])
        _, g_loss_value, = sess.run([g_train_op, losses[gan.g]], feed_dict={Xr: X_mb, Z1:z1})

        # 結果をappend
        loss["dis_loss"].append(d_loss_value)
        loss["gen_loss"].append(g_loss_value)
        print("epoch:" + str(epoch))
        # グラフの描画（余裕があったら）
        if epoch % display_epoch == 0:
            save_metrics(loss, epoch)

        if epoch % param_save_epoch == 0:
            path = "model"
            if not os.path.isdir(path):
                os.makedirs(path)

            saver.save(sess, "./model/dcgan_model" + str(epoch) + ".ckpt")

In [ ]:
import numpy as np

a = np.arange(10)
print(a)
print(a.shape)

b = a - 1
print(b)
print(b.shape)
print(np.abs(b))
#np.abs
#tf.pow((tf.abs(delta)-1), 2))

In [ ]:
a = [[[0],[1],[2]],
     [[0],[1],[2]],
     [[0],[1],[2]],
     [[0],[1],[2]]]

b = tf.reduce_sum(a, axis=0)

with tf.Session() as sess:
    c = sess.run(b)
    
print(np.array(a).shape)
print(c)

In [ ]:
import tensorflow as tf
import numpy as np

x = np.ones([32, 32, 32, 3])
y = np.zeros([32, 1, 1, 10])

tf.concat([x, y], 3)

print(x.shape)
#print(y)
print(y.shape)
